In [1]:
import os, glob
import xuleta as xu
import numpy as np
import pylab as plt

### GENERIC MODIS multi-tile processing and mosaicking
* Remember to set the no-data value accordingly.
* The boundaries are set no Northeastern Brazil

#### MCD43 albedo
* White-sky shortwave albedo, band 9
* Black-sky shortwave albedo, band 19

In [ ]:
folder = '/media/denis/seagate/PB/MODIS/mod16a2/'
os.chdir(folder)

os.system('mkdir 1309 1310 1409 1410')
os.system('mv *h13v09* 1309; mv *h13v10* 1310; mv *h14v09* 1409; mv *h14v10* 1410')

In [ ]:
os.chdir('/home/denis/apps/MRT/bin')

os.system('java -jar MRTBatch.jar -d /media/denis/seagate/PB/MODIS/mod16a2/1309 -p /media/denis/seagate/PB/MODIS/mod16a2/m1309.prm; chmod +x mrtbatch; ./mrtbatch; rm mrtbatch; java -jar MRTBatch.jar -d /media/denis/seagate/PB/MODIS/mod16a2/1310 -p /media/denis/seagate/PB/MODIS/mod16a2/m1310.prm; chmod +x mrtbatch; ./mrtbatch; rm mrtbatch; java -jar MRTBatch.jar -d /media/denis/seagate/PB/MODIS/mod16a2/1409 -p /media/denis/seagate/PB/MODIS/mod16a2/m1409.prm; chmod +x mrtbatch; ./mrtbatch; rm mrtbatch; java -jar MRTBatch.jar -d /media/denis/seagate/PB/MODIS/mod16a2/1410 -p /media/denis/seagate/PB/MODIS/mod16a2/m1410.prm; chmod +x mrtbatch; ./mrtbatch; rm mrtbatch; ')

In [ ]:
for i,j,k,l in zip(sorted(glob.glob(folder+"1309/prm/*ET*.tif")),sorted(glob.glob(folder+"1310/prm/*ET*.tif")),
                   sorted(glob.glob(folder+"1409/prm/*ET*.tif")),sorted(glob.glob(folder+"1410/prm/*ET*.tif"))):
    g1 = 'gdal_merge.py -ul_lr -48.7551 -1.0445 -32.3925 -18.3494 -n 32767 -a_nodata 32767 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -of GTiff -o '

    go = folder+i.split('.')[-3]+'.tif '
    g2 = i+' '+j+' '+k+' '+l
    gf = g1+go+g2
    os.system(gf)

In [ ]:
xu.renamedate(folder=folder,oldf="%Y%j",newf='%Y-%m-%d',wts=0)

#### Addressing further issues with no-data

In [ ]:
import xuleta as xu 
import numpy as np
print(folder)

In [ ]:
for i in glob.glob(folder+"*.tif"):
    a,m = xu.TifToArray(i)
    a = np.where(a>30000,32767,a)
    xu.ArrayToTif(a,i.split('/')[-1],Folder=folder,Metadata=m,Type=1)

In [ ]:
for i in glob.glob(folder+"*.tif"):
    a,m = xu.TifToArray(i)
    b = np.where(a>30000,32767,a)
    fn = i.split("/")[-1]
    xu.ArrayToTif(b,Folder=folder,Filename=fn,Metadata=m,Type=1)

In [ ]:
os.chdir(folder)
os.system('for f in *.tif; do gdalwarp -overwrite -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -multi -dstnodata 32767 -of GTiff $f t$f; done')

In [ ]:
folder = '/media/denis/seagate/PB/MODIS/myd17a2h/'
os.chdir(folder)
os.system('rm A*.tif')

In [ ]:
#CLEANING - MAKE SURE IS EVERYTHING OK BEFORE DOING IT 
os.chdir(folder)
os.system('find . -name "*.hdf" -type f -delete')
os.system('rm -rf 1309 1310 1409 1410')

In [ ]:
xu.renamedate(folder, oldf='%Y%j',wts=0)

### REFL, NDVI

In [ ]:
folder = '/media/denis/seagate/PB/MODIS/myd09a1/'

**RUN ZHAODELONG NDVI script for each tile**

In [ ]:
for i,j,k,l in zip(sorted(glob.glob(folder+"t1309/ndvi/*tif")),sorted(glob.glob(folder+"t1310/ndvi/*tif")),
                   sorted(glob.glob(folder+"t1409/ndvi/*tif")),sorted(glob.glob(folder+"t1410/ndvi/*tif"))):
    g1 = 'gdal_merge.py  -n -2 -a_nodata -2 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -of GTiff -o '

    go = folder+i.split('.')[-6]+'.tif '
    g2 = i+' '+j+' '+k+' '+l
    gf = g1+go+g2
    #print(gf)
    os.system(gf)

In [ ]:
folder = '/media/denis/seagate/PB/MODIS/myd09a1/'
for i in glob.glob(folder+"*.tif"):
    a,m = xu.TifToArray(i)
    a = np.where(a<-0.2,-2,a)
    a = a*10000
    xu.ArrayToTif(Array=a,Folder=folder,Filename=i.split("/")[-1],Metadata=m,Type=1)

In [ ]:
os.chdir(folder)
os.system('for f in *.tif; do gdalwarp -ot Int16 -s_srs "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs" -t_srs EPSG:4326 -dstnodata -20000 -r near -of GTiff -te -48.7551 " -18.3494" " -34.729" -1.0445 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -wo OPTIMIZE_SIZE=TRUE $f r$f; done')

## Calculating montly means and maybe other aggregation

In [19]:
import xuleta as xu
import numpy as np
import pylab as plt
import bottleneck as bn
from glob import glob
import os


In [24]:
folder = '/media/denis/seagate/PBv2/mxd16a2_anomalies/out/'
ul = 12
ll = -12
nodata = None

In [25]:
if not os.path.exists(folder+"monthly_mean"):
    os.makedirs(folder+"monthly_mean")
folderout = folder+"monthly_mean/"

l = sorted(glob(folder+"*.tif"))

#list of unique Year-months
ym = sorted(list(set([ii[-14:-7] for n,ii in enumerate(l) if ii not in l[:n]])))

for m in ym:
    #list of files that match the current year-month
    toOpen = [x for x in l if m in x]
    
    arrays = []
    for j in toOpen:
        imagem, meta = xu.TifToArray(j)
        imagem = np.where(imagem > ul, np.nan, imagem)
        imagem = np.where(imagem < ll, np.nan, imagem)
        arrays.append(imagem)
        arrays2 = np.dstack(arrays)
    xu.ArrayToTif(np.nanmean(arrays2, axis=2), m+'.tif', folderout, meta, Type=3)  
    arrays, media, toOpen, arrays2 = None, None, None, None

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice


2002-07.tif saved.
 
2002-08.tif saved.
 
2002-09.tif saved.
 
2002-10.tif saved.
 
2002-11.tif saved.
 
2002-12.tif saved.
 
2003-01.tif saved.
 
2003-02.tif saved.
 
2003-03.tif saved.
 
2003-04.tif saved.
 
2003-05.tif saved.
 
2003-06.tif saved.
 
2003-07.tif saved.
 
2003-08.tif saved.
 
2003-09.tif saved.
 
2003-10.tif saved.
 
2003-11.tif saved.
 
2003-12.tif saved.
 
2004-01.tif saved.
 
2004-02.tif saved.
 
2004-03.tif saved.
 
2004-04.tif saved.
 
2004-05.tif saved.
 
2004-06.tif saved.
 
2004-07.tif saved.
 
2004-08.tif saved.
 
2004-09.tif saved.
 
2004-10.tif saved.
 
2004-11.tif saved.
 
2004-12.tif saved.
 
2005-01.tif saved.
 
2005-02.tif saved.
 
2005-03.tif saved.
 
2005-04.tif saved.
 
2005-05.tif saved.
 
2005-06.tif saved.
 
2005-07.tif saved.
 
2005-08.tif saved.
 
2005-09.tif saved.
 
2005-10.tif saved.
 
2005-11.tif saved.
 
2005-12.tif saved.
 
2006-01.tif saved.
 
2006-02.tif saved.
 
2006-03.tif saved.
 
2006-04.tif saved.
 
2006-05.tif saved.
 
2006-06.tif s

In [27]:
#COMPRESS FILES
os.chdir(folderout)
#-ot Int16
#os.system('for f in *.tif; do gdalwarp -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=9 -multi -of GTiff $f X$f; done')
xu.findreplace(folderout,'X','')